In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta


## MPT - GARCH

In [59]:
# Re-importing the test.csv dataset using the correct date column name

# Path = 'test4.csv'
assets = ['DHR','GILD','ISRG','JNJ','LLY','PFE','REGN','TMO','AZN','MRK'] 
asset = 'DHR'
Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
# Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
# Path = './Evaluation/GCH_1.csv'
GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
GARCH_mpt 

,h1,h2,h3,h4,TrueSD
Date_Forecast,,,,,
2007-12-28,0.040823,0.046994,0.022860,0.032679,0.021739
2008-01-04,0.026175,0.021890,0.023069,0.031169,0.037096
2008-01-11,0.026458,0.016188,0.032649,0.022329,0.045272
2008-01-18,0.026837,0.019938,0.027050,0.016868,0.033410
2008-01-25,0.031670,0.027283,0.025233,0.032141,0.022480
...,...,...,...,...,...
2015-11-27,0.020455,0.026958,0.026176,0.031186,0.011314
2015-12-04,0.023041,0.054805,0.042789,0.045970,0.030199
2015-12-11,0.026475,0.030140,0.020500,0.032568,0.025063


## Monthly Aggregation

In [60]:
# Convert the index to datetime format

# Resample to monthly frequency
df_monthly = GARCH_mpt.resample('M').median()* np.sqrt(4)
df_monthly = df_monthly[['h4','TrueSD']]
df_monthly.head()

,h4,TrueSD
Date_Forecast,,
2007-12-31,0.065359,0.043478
2008-01-31,0.053499,0.070506
2008-02-29,0.046198,0.053232
2008-03-31,0.054741,0.084540
2008-04-30,0.055467,0.073553


## MASE

In [61]:
# Load the updated data
df_updated2 = df_monthly.copy()
df_updated2
def MASE(df_updated):
    # Calculate MAE for each horizon
    mae_h4_updated = np.mean(np.abs(df_updated['h4'] - df_updated['TrueSD']))

    # Calculate MAE of the naive forecast (using last known value)
    mae_naive_updated = np.mean(np.abs(df_updated['TrueSD'].shift(1) - df_updated['TrueSD']))


    # Compute MASE for each horizon
    mase_h4_updated = mae_h4_updated / mae_naive_updated

    return mase_h4_updated

MASE(df_updated2)

1.1922357573026399

In [62]:

Results=pd.DataFrame({'Asset':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    GARCH_mpt= GARCH_mpt.resample('M').median()* np.sqrt(4)
    GARCH_mpt= GARCH_mpt[['h4','TrueSD']]
    IterResults={'Asset': asset, 'h4': MASE(GARCH_mpt)}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)    
    

In [77]:

# Define the MASE function
def MASE(actual_series, forecast_series):
    scaled_error = np.mean(np.abs(forecast_series - actual_series))
    scale = np.mean(np.abs(actual_series.diff().dropna()))
    return scaled_error / scale


# Initialize results DataFrame
Results = pd.DataFrame(columns=['Asset', 'Monthly MASE'])

for asset in assets:
    # Adjusted the path to where you've uploaded the files
    path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    
    # Read the data, assuming the CSV files have a 'Date_Forecast' column
    data = pd.read_csv(path, index_col='Date_Forecast', parse_dates=True)
    
    # Calculate the median of forecasts before resampling
    data_median = data.resample('M').median()

    # Calculate the volatility forecast (if 'h4' is your forecast column)
    volatility_forecast = data_median['h4'] * np.sqrt(4)

    # Assuming 'TrueSD' is the actual observed volatility
    actual_volatility = data_median['TrueSD']
    
    # Calculate MASE for the asset
    mase = MASE(actual_volatility, volatility_forecast)
    
    # Append the results
    Results = Results.append({'Asset': asset, 'Monthly MASE': mase}, ignore_index=True)

# Display the results
print(Results)


  Asset  Monthly MASE
0   DHR      3.571693
1  GILD      3.096440
2  ISRG      2.593165
3   JNJ      5.056627
4   LLY      3.961392
5   PFE      4.240119
6  REGN      2.855495
7   TMO      3.280145
8   AZN      3.580026
9   MRK      2.965306


In [64]:
# Function to calculate MASE for each individual forecast point
def calculate_monthly_MASE(data):
    # Calculating the absolute errors for each forecast point
    absolute_errors = np.abs(data['TrueSD'] - data['h4'])

    # Calculating the scaling factor using training data's actual values
    scale = np.mean(np.abs(data['TrueSD'].diff().dropna()))

    # Calculate and return the individual MASE values
    return absolute_errors / scale

# Create a dictionary to hold the data frames for each asset
monthly_MASE_data = {}

for asset in assets:
    # Set the path to the uploaded files
    path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    
    # Read the data
    data = pd.read_csv(path, index_col='Date_Forecast', parse_dates=True)
    
    # Calculating the volatility forecast (assuming 'h4' is the forecast column)
    data['h4'] = data['h4'] * np.sqrt(4)  # scaling by sqrt(4)

    # Resampling by month and applying the MASE calculation function
    monthly_MASE = data.resample('M').apply(calculate_monthly_MASE)

    # Store the monthly MASE DataFrame in our dictionary
    monthly_MASE_data[asset] = monthly_MASE

# Display the first few rows of each asset's monthly MASE to verify the results
{asset: df.head() for asset, df in monthly_MASE_data.items()}


{'DHR': Date_Forecast  Date_Forecast
 2007-12-31     2007-12-28            NaN
 2008-01-31     2008-01-04       2.445330
                2008-01-11       0.059430
                2008-01-18       0.031711
                2008-01-25       4.049580
 dtype: float64,
 'GILD': Date_Forecast  Date_Forecast
 2007-12-31     2007-12-28            NaN
 2008-01-31     2008-01-04       0.110798
                2008-01-11       1.199050
                2008-01-18       5.334744
                2008-01-25       0.275508
 dtype: float64,
 'ISRG': Date_Forecast  Date_Forecast
 2007-12-31     2007-12-28            NaN
 2008-01-31     2008-01-04       0.053423
                2008-01-11       0.620871
                2008-01-18       1.325430
                2008-01-25       1.000188
 dtype: float64,
 'JNJ': Date_Forecast  Date_Forecast
 2007-12-31     2007-12-28            NaN
 2008-01-31     2008-01-04       3.908951
                2008-01-11       3.015842
                2008-01-18       0.788208
 

In [65]:
# Correcting the aggregation process to handle the data appropriately
aggregated_monthly_MASE = pd.DataFrame()

for asset, MASE_series in monthly_MASE_data.items():
    # Convert the series into a DataFrame for easier processing
    MASE_df = MASE_series.to_frame(name="MASE")
    
    # Remove the multi-index, which seems to be creating complications
    MASE_df = MASE_df.reset_index(level=0, drop=True)
    
    # Group the data by month, calculate the average MASE, and add it to the final DataFrame
    monthly_avg = MASE_df.resample('M').mean()
    aggregated_monthly_MASE[asset] = monthly_avg["MASE"]

# Checking the first few rows of the aggregated DataFrame to ensure the process worked correctly
aggregated_monthly_MASE.head()


,DHR,GILD,ISRG,JNJ,LLY,PFE,REGN,TMO,AZN,MRK
Date_Forecast,,,,,,,,,,
2007-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-31,1.646513,1.730025,0.749978,2.155401,2.370354,3.022296,5.218808,1.916381,0.799855,2.189694
2008-02-29,3.801211,2.528359,1.188354,18.944440,6.292579,6.665604,2.694421,1.623165,1.311285,2.874436
2008-03-31,2.835789,1.758117,0.878357,8.098091,1.099508,2.336556,0.988026,1.148113,1.014877,0.560783
2008-04-30,2.085177,1.242764,0.489290,3.577268,1.818893,4.680047,1.653013,2.201283,3.808931,0.382518


In [66]:
Assest =  ['DHR','GILD','ISRG','JNJ','LLY','PFE','REGN','TMO','AZN','MRK'] 
mask = Results['Asset'].isin(Assest)
Result = Results[mask]
Result.T

,0,1,2,3,4,5,6,7,8,9
Asset,DHR,GILD,ISRG,JNJ,LLY,PFE,REGN,TMO,AZN,MRK
Monthly MASE,3.57169,3.09644,2.59317,5.05663,3.96139,4.24012,2.85549,3.28015,3.58003,2.96531


## RMSE

In [78]:
import numpy as np

def RMSE(actual, predicted):
    """
    Calculate the Root Mean Square Error (RMSE) between actual and predicted values.
    
    Parameters:
    - actual (array-like): The true values.
    - predicted (array-like): The predicted values.
    
    Returns:
    - Root Mean Square Error (RMSE) value.
    """
    return round(np.sqrt(np.mean((actual - predicted)**2)),5)*100


In [79]:
GARCH_mpt

,h4,TrueSD
Date_Forecast,,
2007-12-31,0.063147,0.035885
2008-01-31,0.065868,0.086953
2008-02-29,0.066745,0.070069
2008-03-31,0.061415,0.076703
2008-04-30,0.047317,0.120602
...,...,...
2015-08-31,0.055031,0.092369
2015-09-30,0.062613,0.071281
2015-10-31,0.042050,0.075349


In [80]:

Results=pd.DataFrame({'Asset':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    GARCH_mpt= GARCH_mpt.resample('M').median()* np.sqrt(4)
    GARCH_mpt= GARCH_mpt[['h4','TrueSD']]
    
    IterResults={'Asset': asset, 'h4': RMSE(GARCH_mpt['TrueSD'],GARCH_mpt['h4'])}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)    

In [81]:
Results.T

,0,1,2,3,4,5,6,7,8,9
Asset,DHR,GILD,ISRG,JNJ,LLY,PFE,REGN,TMO,AZN,MRK
h4,3.135,3.2,4.196,2.765,3.569,3.021,4.133,3.664,3.108,3.757


### MAPE

In [71]:
def MAPE(actual, predicted):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between actual and predicted values.
    
    Parameters:
    - actual (array-like): The true values.
    - predicted (array-like): The predicted values.
    
    Returns:
    - Mean Absolute Percentage Error (MAPE) value.
    """
    # Avoid division by zero by adding a small value to the denominator
    return round(np.mean(np.abs((actual - predicted) / (actual + 1e-10))) * 100,2)




In [72]:

Results=pd.DataFrame({'Asset':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    # Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    GARCH_mpt= GARCH_mpt.resample('M').median()* np.sqrt(4)
    GARCH_mpt= GARCH_mpt[['h4','TrueSD']]
    
    IterResults={'Asset': asset, 'h4': MAPE(GARCH_mpt['TrueSD'],GARCH_mpt['h4'])}
    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)   
Results.T

,0,1,2,3,4,5,6,7,8,9
Asset,DHR,GILD,ISRG,JNJ,LLY,PFE,REGN,TMO,AZN,MRK
h4,42.41,31.77,40.78,73.87,48.14,45.28,30.4,39.17,37.71,50.16


## RMPSE

In [73]:
def RMPSE(actual, predicted):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between actual and predicted values.
    
    Parameters:
    - actual (array-like): The true values.
    - predicted (array-like): The predicted values.
    
    Returns:
    - Mean Absolute Percentage Error (MAPE) value.
    """
    # Avoid division by zero by adding a small value to the denominator
    return round((np.sqrt(np.mean(np.square((actual - predicted) / actual)))) * 100,2)

In [74]:

Results=pd.DataFrame({'Asset':[], 'h4': []})
for asset in assets:
    Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/MPT-GARCH/final/5_MTL_GARCH_{}.csv'.format(asset)
    # Path = '/Users/lesegomatojane/Documents/MIT807/Muliti-transformer/MultiTransformer-master/MultiTransformer/Train/assets/Drop001/GARCH/GARCH_{}.csv'.format(asset)
    GARCH_mpt = pd.read_csv(Path, index_col='Date_Forecast', parse_dates=True)
    
    GARCH_mpt= GARCH_mpt.resample('M').median()* np.sqrt(4)
    GARCH_mpt= GARCH_mpt[['h4','TrueSD']]
    
    IterResults={'Asset': asset, 'h4': RMPSE(GARCH_mpt['TrueSD'],GARCH_mpt['h4'])}

    
    IterResults_df = pd.DataFrame(IterResults,index =[0])
    Results= Results.append(IterResults_df, ignore_index=True)   
Results.T

,0,1,2,3,4,5,6,7,8,9
Asset,DHR,GILD,ISRG,JNJ,LLY,PFE,REGN,TMO,AZN,MRK
h4,59.38,39.53,59.82,107.15,69.73,58.86,39.81,53.99,49.75,63.48


In [75]:
GARCH_mpt

,h4,TrueSD
Date_Forecast,,
2007-12-31,0.063147,0.035885
2008-01-31,0.065868,0.086953
2008-02-29,0.066745,0.070069
2008-03-31,0.061415,0.076703
2008-04-30,0.047317,0.120602
...,...,...
2015-08-31,0.055031,0.092369
2015-09-30,0.062613,0.071281
2015-10-31,0.042050,0.075349


# Aggregate Monthly

In [76]:
import plotly.graph_objects as go

# Create the interactive line plot using Plotly
fig = go.Figure()


fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h4'], mode='lines+markers', name='MASE_h4', marker=dict(symbol='cross')))

# Add a horizontal line at MASE=1 for reference
fig.add_shape(
    type='line',
    line=dict(dash='dash', color='black'),
    x0=mase_direct_monthly_df.index[0],
    x1=mase_direct_monthly_df.index[-1],
    y0=1,
    y1=1
)

# Update the layout for transparent background, black fonts, and other stylistic choices
fig.update_layout(
    title='MASE values for Different Forecast Horizons over Time',
    xaxis_title='Date',
    yaxis_title='MASE',
    template="none",
    yaxis=dict(type='log', gridcolor='gray', tickcolor='gray', tickfont=dict(color='black', size=12)),
    xaxis=dict(gridcolor='gray', tickcolor='gray', tickfont=dict(color='black', size=12)),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent background for the plot area
    paper_bgcolor='rgba(0,0,0,0)',   # Transparent background for the whole figure
    font=dict(color='black', size=12, family='Sans Serif'),
    legend=dict(font=dict(color='black', size=15, family='Sans Serif'))
)

fig.show()


NameError: name 'mase_direct_monthly_df' is not defined

In [ ]:
import plotly.graph_objects as go

# Create the interactive line plot using Plotly
fig = go.Figure()

# Add lines for each forecast horizon with different markers
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h1'], mode='lines+markers', name='MASE_h1', marker=dict(symbol='circle')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h2'], mode='lines+markers', name='MASE_h2', marker=dict(symbol='square')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h3'], mode='lines+markers', name='MASE_h3', marker=dict(symbol='diamond')))
fig.add_trace(go.Scatter(x=mase_direct_monthly_df.index, y=mase_direct_monthly_df['MASE_h4'], mode='lines+markers', name='MASE_h4', marker=dict(symbol='cross')))

# Add a horizontal line at MASE=1 for reference
fig.add_shape(
    type='line',
    line=dict(dash='dash', color='white'),
    x0=mase_direct_monthly_df.index[0],
    x1=mase_direct_monthly_df.index[-1],
    y0=1,
    y1=1
)

# Update the layout for transparent background and other stylistic choices
fig.update_layout(
    title='MASE values for Different Forecast Horizons over Time',
    xaxis_title='Date',
    yaxis_title='MASE',
    template="none",
    yaxis=dict(type='log', gridcolor='gray', tickcolor='gray', tickfont=dict(color='black')),
    xaxis=dict(gridcolor='gray', tickcolor='gray', tickfont=dict(color='black')),
    plot_bgcolor='rgba(0,0,0,0)',     # Transparent background for the plot area
    paper_bgcolor='rgba(0,0,0,0)',   # Transparent background for the whole figure
    font=dict(color='black'),
    legend=dict(font=dict(color='black'))
)

fig.show()


In [ ]:
import pandas as pd
from scipy.stats import skew, kurtosis
import numpy as np

def round_to_sf(number, sf):
    """Round number to specified significant figures."""
    return round(number, sf - int(np.floor(np.log10(abs(number)))) - 1)

def compute_summary_statistics(stock_data):
    """
    Compute summary statistics for weekly returns of multiple stocks.

    Parameters:
    - stock_data: DataFrame where columns are stock tickers and rows represent weekly returns.

    Returns:
    - summary_df: DataFrame with summary statistics for each stock.
    """
    # Initializing an empty DataFrame for summary statistics
    summary_df = pd.DataFrame(index=['Min', 'Max', 'Mean', 'Std', 'Skewness', 'Kurtosis'])

    # Looping through each stock and computing the statistics
    for stock in stock_data.columns:
        stock_returns = stock_data[stock].dropna()  # Remove NaN values
        stats = {
            'Min': round_to_sf(stock_returns.min(), 4),
            'Max': round_to_sf(stock_returns.max(), 4),
            'Mean': round_to_sf(stock_returns.mean(), 4),
            'Std': round_to_sf(stock_returns.std(), 4),
            'Skewness': round_to_sf(skew(stock_returns), 4),
            'Kurtosis': round_to_sf(kurtosis(stock_returns), 4)
        }
        summary_df[stock] = stats.values()

    return summary_df

# Example usage:
# Assuming 'weekly_returns_df' is your DataFrame containing weekly returns for multiple stocks
# summary_statistics = compute_summary_statistics(weekly_returns_df)
# print(summary_statistics)


In [ ]:
import yfinance as yf

Start='2008-01-01'; End='2015-12-31'; 
asset = ['^GSPC','AAPL' ,'MSFT' ,'NVDA', 'JNJ', 'NVS' ,'JPM' ,'GS' ,'AMZN']
Database=yf.download(asset,Start, End, progress=False).resample('W-FRI').last()['Adj Close']
Database
# compute_summary_statistics(Database)

,AAPL,AMZN,GS,JNJ,JPM,MSFT,NVDA,NVS,^GSPC
Date,,,,,,,,,
2008-01-04,5.458065,4.439500,155.722641,41.469311,27.268015,25.068100,6.880692,27.101757,1411.630005
2008-01-11,5.234954,4.054000,154.795792,42.754204,27.221386,24.725409,6.204093,28.675022,1401.020020
2008-01-18,4.891492,3.988000,145.815262,41.752739,26.375301,24.069164,5.697215,26.483334,1325.189941
2008-01-25,3.941144,3.880000,149.317734,39.340416,29.073448,24.018127,5.722443,24.855644,1330.609985
2008-02-01,4.054520,3.731500,162.121750,39.907272,32.144684,22.202549,6.160515,24.949642,1395.420044
...,...,...,...,...,...,...,...,...,...
2015-12-04,27.126707,33.632000,163.150940,83.646179,54.160393,49.786156,8.237807,57.215130,2091.689941
2015-12-11,25.793507,32.007500,151.618118,82.614311,51.112938,48.138779,7.930265,56.087013,2012.369995
2015-12-18,24.164038,33.207001,150.699265,82.833694,51.376202,48.201115,7.844831,56.252914,2005.550049


In [ ]:
Database.to_csv('ADF_test.csv')

In [ ]:
compute_summary_statistics(Database).T

,Min,Max,Mean,Std,Skewness,Kurtosis
AAPL,2.496,29.940,13.420,7.755,0.4380,-0.8865
AMZN,1.893,34.450,11.430,6.893,0.9750,0.8896
GS,41.880,181.800,123.900,28.730,-0.4008,-0.4019
JNJ,31.340,85.630,54.880,16.360,0.6116,-1.2350
JPM,11.040,54.820,33.790,9.614,0.4976,-0.7281
MSFT,11.420,50.140,25.860,8.561,0.8842,-0.1600
NVDA,1.463,8.238,3.752,1.226,0.9633,1.3070
NVS,18.490,70.210,39.060,14.220,0.6825,-0.8439
^GSPC,683.400,2127.000,1450.000,382.100,0.3101,-1.0230


In [ ]:
import numpy as np

# Redefining the given functions and the example data
def ReturnCalculation(Database, Lag):
    # Dummy return calculation function for the example
    # In a real scenario, this function calculates the returns based on the given lag
    return Database, list(range(len(Database)))

def SDCalculation(DailyReturns, LagSD):
    dimension = len(DailyReturns)
    dif = LagSD
    Out = np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i] = np.std(DailyReturns[i:i+LagSD], ddof=1)
    return np.append(np.repeat(np.nan, dif), Out)

def TrueSDCalculation(DailyReturns, LagSD):
    dimension = len(DailyReturns)
    dif = LagSD
    Out = np.zeros([dimension-dif+1])
    for i in range(dimension-dif+1):
        Out[i] = np.std(DailyReturns[i:i+LagSD], ddof=1)
    return np.append(Out, np.repeat(np.nan, dif-1))

# Example data
DailyReturns = [1, 2, 4.5, 7.3, 3.1, 6.1,3, 7.1, 4.8, 6.9, 10]
LagSD = 3

# Calculations
sd_values = SDCalculation(DailyReturns, LagSD)
true_sd_values = TrueSDCalculation(DailyReturns, LagSD)

sd_values, true_sd_values


(array([       nan,        nan,        nan, 1.80277564, 2.65141472,
        2.13853532, 2.16333077, 1.76162803, 2.13775583, 2.05507502,
        1.27410099]),
 array([1.80277564, 2.65141472, 2.13853532, 2.16333077, 1.76162803,
        2.13775583, 2.05507502, 1.27410099, 2.61597655,        nan,
               nan]))